In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [22]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository


## Load Data

In [24]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv('LOAD_DATA_FROM_DWH'))

false


In [25]:
data_loader=DataLoader()
ppm_repo = PortpropMatricesRepository(data_loader=data_loader)
ports_repo = PortfoliosRepository(data_loader=data_loader)
rebalancer_repo = RebalancerRepository(data_loader=data_loader)

In [26]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-08-31"
)
client_styles = ports_repo.load_client_style(
    as_of_date="2025-08-31"
)

In [27]:
ports_ref_table = {
            'product_mapping' : ports_repo.load_product_mapping(),
            'product_underlying' : ports_repo.load_product_underlying(),
}

In [28]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [29]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [30]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(client_out_enriched, client_styles, column_mapping=['as_of_date', 'customer_id'])
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer()
rb.set_ref_tables(rb_ref_dict)


## Portfolio (Service)

In [31]:
## get list of all customer ids
print(port_service.get_all_customer_ids())

[24149, 27766, 28359, 23349, 36170, 27144, 18051, 30961, 35258, 45708, 45701, 50745, 51637, 61749, 58055, 72106, 74374, 76048, 76011, 72660, 86279, 94903, 57414, 96146, 53015, 23905, 35261, 30030, 103542, 107010, 52747, 27547, 14154, 21427, 12346, 49474, 54552, 19583, 48844, 30174, 31843, 35059, 26925, 30084, 52903, 28587, 28508, 11838, 18550, 23356, 29901, 32263, 12883, 109086, 80627, 17739, 16906, 10052, 12001, 28999, 24287, 33624, 17533, 26352, 33901, 25512, 12594, 23791, 33516, 39036, 28624, 24845, 33713, 25735, 37403, 37679, 51513, 57478, 58082, 58556, 62864, 79790, 80115, 88167, 90407, 90529, 91843, 93204, 88598, 99370, 22207, 26447, 102165, 103356, 106679, 110989, 21373, 21228, 22556, 14204, 21986, 93776, 37801, 62708, 31771, 26339, 77908, 24760, 26184, 13139, 34979, 33416, 103969, 36363, 23801, 31493, 22560, 21010, 13634, 31626, 13418, 22997, 84563, 24268, 32142, 15220, 26338, 32926, 35510, 36916, 22242, 28760, 29950, 24563, 33182, 18218, 46512, 55606, 35978, 56101, 58697, 5055

In [32]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=12560)

In [33]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
399,12560,2025-08-31,B00174068,NTL26NA,TRADE,L,THB,NTL26NA,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,2.065495e+06,1147,AA_FI,0.012518
400,12560,2025-08-31,B00174420,CPF23PA,TRADE,L,THB,CPF23PA,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,2.138060e+06,1147,AA_FI,0.012958
453,12560,2025-08-31,M00299362,KKP SIB-USD,TRADE,L,USD,KKP SIB-USD,Mutual Fund,Fixed Income,...,True,0.048,False,<NA>,Not Top-Pick,<NA>,4.980829e+03,1147,AA_FI,0.000030
1375,12560,2025-08-31,B00138028,MINT23PA,TRADE,L,THB,MINT23PA,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,4.133600e+06,1147,AA_FI,0.025052
1379,12560,2025-08-31,B00164409,CPAXT289A,TRADE,L,THB,CPAXT289A,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,2.097467e+06,1147,AA_FI,0.012712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18416,12560,2025-08-31,T00320952,XS3102117291,GIS,L,USD,SHFIN20250630A,Structured Note,Global Equity,...,True,0.077,False,<NA>,Not Top-Pick,<NA>,1.599890e+06,1147,AA_GE,0.009696
19216,12560,2025-08-31,B00221075,IVL24PA,TRADE,L,THB,IVL24PA,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,2.327808e+06,1147,AA_FI,0.014108
19219,12560,2025-08-31,B00304257,USY0616GAA14,GIS,L,USD,USY0616GAA14YQ4728834 corp,Fixed Income,Fixed Income,...,True,0.025,False,<NA>,Not Top-Pick,<NA>,6.560369e+06,1147,AA_FI,0.039759
19291,12560,2025-08-31,M00232288,KKP PGE RMF-UH,TRADE,L,THB,KKP PGE RMF-UH,Mutual Fund,Global Equity,...,True,0.077,False,<NA>,Top-Pick,RMF,5.187173e+06,1147,AA_GE,0.031437


In [34]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,1147,0.060932,0.041909,0.400828,0.390025,0.106306


In [35]:
# get model allocation 
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,1147,Moderate High Risk,Medium to Moderate High Risk,0.13,0.06,0.27,0.45,0.05


## Healthscore

In [36]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm,hs)

In [37]:
health_score['health_score'].values[0]

np.float64(8.0)

## Rebalancer

In [38]:
rb.rebalance(port, ppm, hs)

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,value,weight,flag,expected_weight,action,amount
0,1,1,1147,S00087551,BTSGIF,TRADE,L,THB,3545000.0,0.021485,not_monitored_product,0.000000,sell,-3.545000e+06
1,2,1,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,3.545000e+06
2,3,2,1147,S00237377,VAYU1,TRADE,L,THB,15900000.0,0.096362,not_monitored_product,0.000000,sell,-1.590000e+07
3,4,2,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,1.590000e+07
4,5,3,1147,DTHB00000000,KKPBAPLUSTHB,TRADE,L,THB,0.0,0.000000,discretionary_buy,0.090553,buy,1.494139e+07
5,6,3,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,-1.494139e+07


In [39]:
port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,1147,B00138028,MINT23PA,TRADE,L,THB,4.133600e+06,MINT23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.025,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.025052
1,1147,B00138680,TUC282A,TRADE,L,THB,2.026721e+06,TUC282A,Fixed Income,Fixed Income,...,True,N/A,True,0.025,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.012283
2,1147,B00143744,BGRIM23PA,TRADE,L,THB,2.099210e+06,BGRIM23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.025,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.012722
3,1147,B00145295,LOTUSS284A,TRADE,L,THB,3.194455e+06,LOTUSS284A,Fixed Income,Fixed Income,...,True,N/A,True,0.025,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.019360
4,1147,B00151378,SIRI25DA,TRADE,L,THB,2.004395e+06,SIRI25DA,Fixed Income,Fixed Income,...,True,N/A,True,0.040,False,<NA>,Not Top-Pick,<NA>,AA_FI,0.012148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1147,T00281178,XS3004384742,GIS,L,USD,9.804772e+05,SHFIN20250221A,Structured Note,Global Equity,...,True,N/A,True,0.077,False,<NA>,Not Top-Pick,<NA>,AA_GE,0.005942
71,1147,T00305836,XS2993068878,GIS,L,USD,2.617949e+06,SHFIN20250401A,Structured Note,Global Equity,...,True,N/A,True,0.077,False,<NA>,Not Top-Pick,<NA>,AA_GE,0.015866
72,1147,T00310048,XS3048279049,GIS,L,USD,1.927777e+06,SHFIN20250428A,Structured Note,Global Equity,...,True,N/A,True,0.077,False,<NA>,Not Top-Pick,<NA>,AA_GE,0.011683
73,1147,T00320952,XS3102117291,GIS,L,USD,1.599890e+06,SHFIN20250630A,Structured Note,Global Equity,...,True,N/A,True,0.077,False,<NA>,Not Top-Pick,<NA>,AA_GE,0.009696
